## Contributions

In [4]:
import os
import json
import torch
import random
import math
import re
import torch.nn as nn
import torch.distributed as dist
from pathlib import Path
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from torch.optim import Optimizer
from torch.optim import AdamW
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data.distributed import DistributedSampler
from pathlib import Path
from tqdm import tqdm
 
class myDataset(Dataset):
  def __init__(self, data_dir, segment_len=128):
    self.data_dir = data_dir
    self.segment_len = segment_len
 
    # Load the mapping from speaker neme to their corresponding id. 
    mapping_path = Path(data_dir) / "mapping.json"
    mapping = json.load(mapping_path.open())
    self.speaker2id = mapping["speaker2id"]
 
    # Load metadata of training data.
    metadata_path = Path(data_dir) / "metadata.json"
    metadata = json.load(open(metadata_path))["speakers"]
 
    # Get the total number of speaker.
    self.speaker_num = len(metadata.keys())
    self.data = []
    for speaker in metadata.keys():
      for utterances in metadata[speaker]:
        self.data.append([utterances["feature_path"], self.speaker2id[speaker]])
 
  def __len__(self):
    return len(self.data)
 
  def __getitem__(self, index):
    feat_path, speaker = self.data[index]
    # Load preprocessed mel-spectrogram.
    mel = torch.load(os.path.join(self.data_dir, feat_path))
 
    # Segmemt mel-spectrogram into "segment_len" frames.
    if len(mel) > self.segment_len:
      # Randomly get the starting point of the segment.
      start = random.randint(0, len(mel) - self.segment_len)
      # Get a segment with "segment_len" frames.
      mel = torch.FloatTensor(mel[start:start+self.segment_len])
    else:
      mel = torch.FloatTensor(mel)
    # Turn the speaker id into long for computing loss later.
    speaker = torch.FloatTensor([speaker]).long()
    return mel, speaker
 
  def get_speaker_number(self):
    return self.speaker_num


def collate_batch(batch):
  # Process features within a batch.
  """Collate a batch of data."""
  mel, speaker = zip(*batch)
  # Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
  mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
  # mel: (batch size, length, 40)
  return mel, torch.FloatTensor(speaker).long()

def get_dataloader(data_dir, batch_size, n_workers):
  """Generate dataloader"""
  dataset = myDataset(data_dir)
  speaker_num = dataset.get_speaker_number()
  # Split dataset into training dataset and validation dataset
  trainlen = int(0.9 * len(dataset))
  lengths = [trainlen, len(dataset) - trainlen]
  trainset, validset = random_split(dataset, lengths)
  train_loader = DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=collate_batch,
  )
  valid_loader = DataLoader(
    validset,
    batch_size=batch_size,
    num_workers=n_workers,
    drop_last=True,
    pin_memory=True,
    collate_fn=collate_batch,
  )
  return train_loader, valid_loader, speaker_num

def model_fn(batch, model, criterion, device):
  """Forward a batch through the model."""

  mels, labels = batch
  mels = mels.to(device)
  labels = labels.to(device)

  outs = model(mels)

  loss = criterion(outs, labels)

  # Get the speaker id with highest probability.
  preds = outs.argmax(1)
  # Compute accuracy.
  accuracy = torch.mean((preds == labels).float())

  return loss, accuracy

class Classifier(nn.Module):
  def __init__(self, d_model=80, n_spks=600, dropout=0.1):
    super().__init__()

    self.prenet = nn.Linear(40, d_model)
    self.encoder_layer = nn.TransformerEncoderLayer(
      d_model=d_model, dim_feedforward=512, nhead=2
    )
    self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=4)  # Increased num_layers

    self.pred_layer = nn.Sequential(
      nn.Linear(d_model, 2 * d_model),  # Added more complexity to the prediction layers
      nn.ReLU(),
      nn.Dropout(dropout),
      nn.Linear(2 * d_model, d_model),  # Second linear layer
      nn.ReLU(),
      nn.Dropout(dropout),
      nn.Linear(d_model, n_spks),
    )

  def forward(self, mels):
    out = self.prenet(mels)
    out = out.permute(1, 0, 2)
    out = self.encoder(out)
    out = out.transpose(0, 1)
    stats = out.mean(dim=1)

    out = self.pred_layer(stats)
    return out
  

def get_cosine_schedule_with_warmup(
  optimizer: Optimizer,
  num_warmup_steps: int,
  num_training_steps: int,
  num_cycles: float = 0.5,
  last_epoch: int = -1,
):
  def lr_lambda(current_step):
    # Warmup
    if current_step < num_warmup_steps:
      return float(current_step) / float(max(1, num_warmup_steps))
    # decadence
    progress = float(current_step - num_warmup_steps) / float(
      max(1, num_training_steps - num_warmup_steps)
    )
    return max(
      0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
    )

  return LambdaLR(optimizer, lr_lambda, last_epoch)

def valid(dataloader, model, criterion, device): 
  """Validate on validation set."""

  model.eval()
  running_loss = 0.0
  running_accuracy = 0.0
  pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

  for i, batch in enumerate(dataloader):
    with torch.no_grad():
      loss, accuracy = model_fn(batch, model, criterion, device)
      running_loss += loss.item()
      running_accuracy += accuracy.item()

    pbar.update(dataloader.batch_size)
    pbar.set_postfix(
      loss=f"{running_loss / (i+1):.2f}",
      accuracy=f"{running_accuracy / (i+1):.2f}",
    )

  pbar.close()
  model.train()

  return running_accuracy / len(dataloader)

def parse_args():
  """arguments"""
  config = {
    "data_dir": "../Dataset",
    "save_path": "model.ckpt",
    "batch_size": 32,
    "n_workers": 0,
    "valid_steps": 2000,
    "warmup_steps": 1000,
    "save_steps": 4000,
    "total_steps": 350000,
  }

  return config

def find_highest_iter_file(directory):
    ckpt_files = os.listdir(directory)
    max_iter = 0
    max_file = ""

    for file in ckpt_files:
        if file.endswith(".ckpt"):
            iter_no = int(re.findall("\d+", file)[0])
            if iter_no > max_iter:
                max_iter = iter_no
                max_file = file
    return max_file, max_iter

def main(data_dir, save_path, batch_size, n_workers, valid_steps, warmup_steps, total_steps, save_steps):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!")

  train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
  train_iterator = iter(train_loader)
  highest_model_name, start_step = find_highest_iter_file(".")
  model = Classifier(n_spks=speaker_num).to(device)
  
  if highest_model_name:
    model.load_state_dict(torch.load(highest_model_name))
    print(f"[Info]: Loaded pre-trained model: {highest_model_name}")
  
  criterion = nn.CrossEntropyLoss()
  optimizer = AdamW(model.parameters(), lr=1e-3)
  scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)

  best_accuracy = -1.0
  best_state_dict = None

  pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit="step")

  for step in range(start_step, total_steps):
    try:
      batch = next(train_iterator)
    except StopIteration:
      train_iterator = iter(train_loader)
      batch = next(train_iterator)
      
    loss, accuracy = model_fn(batch, model, criterion, device)
    batch_loss = loss.item()
    batch_accuracy = accuracy.item()

    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

    adj_step = step + 1

    pbar.update()
    pbar.set_postfix(
      loss=f"{batch_loss:.2f}",
      accuracy=f"{batch_accuracy:.2f}",
      step=adj_step,
    )

    if (adj_step) % valid_steps == 0:
      pbar.close()

      valid_accuracy = valid(valid_loader, model, criterion, device)
      if valid_accuracy > best_accuracy:
        best_accuracy = valid_accuracy
        best_state_dict = model.state_dict()

      pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit="step")

    if (adj_step) % save_steps == 0 and best_state_dict is not None:
      checkpoint_path = f"model_{adj_step}_{best_accuracy:.4f}.ckpt"
      torch.save(best_state_dict, checkpoint_path)
      pbar.write(f"Step {adj_step}, best model saved. (accuracy={best_accuracy:.4f})")

  pbar.close()

if __name__ == "__main__":
  main(**parse_args())

Train:  21% 412/2000 [01:09<04:27,  5.93step/s, accuracy=0.03, loss=6.31, step=412]


[Info]: Use cuda now!


Train:   7% 133/2000 [00:10<02:22, 13.06step/s, accuracy=0.00, loss=6.37, step=133]
Train: 100% 2000/2000 [02:03<00:00, 16.15step/s, accuracy=0.06, loss=4.53, step=2000]A
Valid: 100% 4832/4836 [00:10<00:00, 445.31 uttr/s, accuracy=0.11, loss=4.51]
Train: 100% 2000/2000 [00:46<00:00, 42.57step/s, accuracy=0.19, loss=3.59, step=4000]
Valid: 100% 4832/4836 [00:02<00:00, 1850.15 uttr/s, accuracy=0.23, loss=3.61]
                                     
Train:   0% 0/2000 [00:00<?, ?step/s]                                              
Train:   0% 8/2000 [00:00<00:54, 36.76step/s, accuracy=0.16, loss=3.56, step=4008]]

Step 4000, best model saved. (accuracy=0.2326)


Train: 100% 2000/2000 [00:47<00:00, 42.36step/s, accuracy=0.25, loss=3.50, step=6000]
Valid: 100% 4832/4836 [00:02<00:00, 1902.75 uttr/s, accuracy=0.29, loss=3.24]
Train: 100% 2000/2000 [00:47<00:00, 42.29step/s, accuracy=0.25, loss=3.13, step=8000]
Valid: 100% 4832/4836 [00:02<00:00, 1877.90 uttr/s, accuracy=0.36, loss=2.87]
                                     
Train:   0% 0/2000 [00:00<?, ?step/s]                                              
Train:   0% 8/2000 [00:00<00:54, 36.25step/s, accuracy=0.25, loss=3.28, step=8008]]

...

Train:  58% 1162/2000 [00:26<00:19, 43.29step/s, accuracy=0.97, loss=0.16, step=285162]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Train: 100% 2000/2000 [00:45<00:00, 44.31step/s, accuracy=0.91, loss=0.32, step=296000]
Valid: 100% 4832/4836 [00:02<00:00, 2078.46 uttr/s, accuracy=0.83, loss=0.94]
                                     
Train:   0% 0/2000 [00:00<?, ?step/s]                                                
Train:   0% 8/2000 [00:00<00:50, 39.37step/s, accuracy=0.94, loss=0.15, step=296008]]

Step 296000, best model saved. (accuracy=0.8262)


Train:  83% 1657/2000 [00:37<00:08, 42.69step/s, accuracy=0.97, loss=0.10, step=297657]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Train: 100% 2000/2000 [00:44<00:00, 44.92step/s, accuracy=0.97, loss=0.11, step=310000]
Valid: 100% 4832/4836 [00:02<00:00, 2065.81 uttr/s, accuracy=0.83, loss=0.92]
Train:  68% 1355/2000 [00:30<00:13, 46.29step/s, accuracy=0.88, loss=0.33, step=311355]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Train: 100

Step 324000, best model saved. (accuracy=0.8332)


Train:  55% 1106/2000 [00:25<00:19, 45.48step/s, accuracy=0.94, loss=0.14, step=325106]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Train: 100% 2000/2000 [00:44<00:00, 45.30step/s, accuracy=0.94, loss=0.32, step=338000]
Valid: 100% 4832/4836 [00:02<00:00, 2074.55 uttr/s, accuracy=0.83, loss=0.90]
Train:  36% 716/2000 [00:15<00:27, 46.75step/s, accuracy=0.91, loss=0.43, step=338716]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Dataset of inference

In [10]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
  def __init__(self, data_dir):
    testdata_path = Path(data_dir) / "testdata.json"
    metadata = json.load(testdata_path.open())
    self.data_dir = data_dir
    self.data = metadata["utterances"]

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    utterance = self.data[index]
    feat_path = utterance["feature_path"]
    mel = torch.load(os.path.join(self.data_dir, feat_path))

    return feat_path, mel


def inference_collate_batch(batch):
  """Collate a batch of data."""
  feat_paths, mels = zip(*batch)

  return feat_paths, torch.stack(mels)


## Main funcrion of Inference

In [18]:
import json
import csv
import glob
from pathlib import Path
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
  """arguments"""
  config = {
    "data_dir": "../Dataset",
    "model_path": "./model.ckpt",
    "output_path": "./output.csv",
  }

  return config


def main(
  data_dir,
  model_path,
  output_path,
):
  """Main function."""
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!")

  mapping_path = Path(data_dir) / "mapping.json"
  mapping = json.load(mapping_path.open())

  dataset = InferenceDataset(data_dir)
  dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    drop_last=False,
    num_workers=0,
    collate_fn=inference_collate_batch,
  )
  print(f"[Info]: Finish loading data!",flush = True)

  speaker_num = len(mapping["id2speaker"])
  model = Classifier(n_spks=speaker_num).to(device)
  # new code to automatically find and load the best model
  list_of_files = glob.glob(f"./*.ckpt")  # get list of all model files
  print(list_of_files)
  latest_file = max(list_of_files, key=lambda x: float(x.split('_')[2].split('.')[1]))  # get the best model file
  print(f"Loading from file {latest_file}")
  model.load_state_dict(torch.load(latest_file))
  model.eval()
  print(f"[Info]: Finish creating model!",flush = True)

  results = [["Id", "Category"]]
  for feat_paths, mels in tqdm(dataloader):
    with torch.no_grad():
      mels = mels.to(device)
      outs = model(mels)
      preds = outs.argmax(1).cpu().numpy()
      for feat_path, pred in zip(feat_paths, preds):
        results.append([feat_path, mapping["id2speaker"][str(pred)]])
  
  with open(output_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(results)


if __name__ == "__main__":
  main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
['./model_52000_0.7163.ckpt', './model_36000_0.5676.ckpt', './model_16000_0.5178.ckpt', './model_48000_0.7061.ckpt', './model_8000_0.4392.ckpt', './model_4000_0.3148.ckpt', './model_60000_0.7305.ckpt', './model_20000_0.5799.ckpt', './model_40000_0.6861.ckpt', './model_56000_0.7305.ckpt', './model_24000_0.6010.ckpt', './model_12000_0.4998.ckpt', './model_68000_0.7417.ckpt', './model_28000_0.6231.ckpt', './model_44000_0.6968.ckpt', './model_32000_0.6527.ckpt', './model_64000_0.7394.ckpt', './model_36000_0.6571.ckpt']
Loading from file ./model_68000_0.7417.ckpt
[Info]: Finish creating model!



 56%|██████████████████████████████████████████████▋                                    | 3744/6657 [00:12<00:08, 333.94it/s]


100%|███████████████████████████████████████████████████████████████████████████████████| 6657/6657 [00:21<00:00, 304.11it/s]
